# Mundo de Wumpus

    primeiro pegaremos o algoritmo implementado das aulas passadas

In [ ]:
import time
from collections import deque
import numpy as np
import random

In [174]:
# Inicialização do Q-table

# Definindo o ambiente do mundo de Wumpus
# 0 = vazio, 1 = poço, 2 = Wumpus, 3 = ouro
# O agente começa na posição (0, 0)

ambiente = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [2, 0, 1, 0],
    [0, 0, 1, 3]
]

acoes = ['N', 'S', 'L', 'O', 'atirar_N', 'atirar_S', 'atirar_L', 'atirar_O']
q_table = {}
gamma = 0.8
alpha = 0.8
epsilon = 0.2
tamanho = 4
episodios = 1000


In [175]:
def estado_para_str(pos):
    return str(pos[0]) + '_' + str(pos[1])

def get_action(state):
    if state not in q_table:
        q_table[state] = {a: 0 for a in acoes}
    if random.random() < epsilon:
        return random.choice(acoes)
    return max(q_table[state], key=q_table[state].get)

def mover(pos, acao):
    x, y = pos
    if acao == 'N':
        x = max(0, x - 1)
    elif acao == 'S':
        x = min(tamanho - 1, x + 1)
    elif acao == 'L':
        y = min(tamanho - 1, y + 1)
    elif acao == 'O':
        y = max(0, y - 1)
    return (x, y)

def atirar(pos, direcao):
    x, y = pos
    if direcao == 'N':
        for i in range(x-1, -1, -1):
            if ambiente[i][y] == 2:
                ambiente[i][y] = 0
                return True
    elif direcao == 'S':
        for i in range(x+1, tamanho):
            if ambiente[i][y] == 2:
                ambiente[i][y] = 0
                return True
    elif direcao == 'L':
        for j in range(y+1, tamanho):
            if ambiente[x][j] == 2:
                ambiente[x][j] = 0
                return True
    elif direcao == 'O':
        for j in range(y-1, -1, -1):
            if ambiente[x][j] == 2:
                ambiente[x][j] = 0
                return True
    return False

def get_evento(celula):
    if celula == 1:
        return "poco"
    elif celula == 2:
        return "wumpus"
    elif celula == 3:
        return "ouro"
    # As condições de "flecha" e "parede" foram removidas pois estavam incorretas.
    # A recompensa por movimento normal (passo) será tratada no loop.
    return "nada"

def get_recompensa(evento):
    if evento == "ouro":
        return 100
    elif evento == "wumpus" or evento == "poco":
        return -100
    elif evento == "errou flecha" or evento == "parede":
        return -10
    return -1

def update_q_table(state, action, reward, next_state):
    if next_state not in q_table:
        q_table[next_state] = {a: 0 for a in acoes}
    max_q_next = max(q_table[next_state].values())
    q_table[state][action] = q_table[state][action] + alpha * (reward + gamma * max_q_next - q_table[state][action])

In [177]:
# Treinamento (Versão Final Corrigida)
for ep in range(episodios):
    pos = (0, 0)
    state = estado_para_str(pos)
    # Reseta o ambiente a cada novo episódio
    ambiente_episodio = [row[:] for row in ambiente]
    wumpus_morto = False

    for step in range(50):
        if state not in q_table:
            q_table[state] = {a: 0 for a in acoes}

        action = get_action(state)
        recompensa = 0

        # --- Lógica para ATIRAR ---
        if 'atirar' in action:
            direcao = action.split('_')[1]
            acertou = atirar(pos, direcao)

            if acertou:
                recompensa = 50  # Recompensa positiva por matar o Wumpus
                wumpus_morto = True
            else:
                recompensa = -10 # Penalidade por errar a flecha

            # Ao atirar, o agente não se move
            prox_state = estado_para_str(pos)
            update_q_table(state, action, recompensa, prox_state)

        # --- Lógica para MOVER ---
        else:
            nova_pos = mover(pos, action)
            # Usa o ambiente do episódio para obter o evento
            evento = get_evento(ambiente_episodio[nova_pos[0]][nova_pos[1]])

            # Se o wumpus foi morto, a casa dele é segura (recompensa de passo)
            if evento == "wumpus" and wumpus_morto:
                 recompensa = -1
            else:
                 recompensa = get_recompensa(evento)

            prox_state = estado_para_str(nova_pos)
            update_q_table(state, action, recompensa, prox_state)
            pos = nova_pos # Atualiza a posição somente após mover

        state = prox_state

        # Condição de término do episódio
        evento_final = get_evento(ambiente_episodio[pos[0]][pos[1]])
        if evento_final == "ouro" or (evento_final == "wumpus" and not wumpus_morto) or evento_final == "poco":
            break

print("Treinamento concluído!")

Treinamento concluído!


In [178]:
# Bloco de Execução Final
print("Execução do agente treinado:")
pos = (0, 0)
state = estado_para_str(pos)
path = [pos]
wumpus_morto = False # Supondo que você queira rastrear isso também na execução

# --- INÍCIO DA CORREÇÃO ---
# Verifica a condição inicial ANTES de iniciar o loop de ações
evento_inicial = get_evento(ambiente[pos[0]][pos[1]])
if evento_inicial == "ouro":
    print("Ouro encontrado na posição inicial!")
else:
    # O loop só é executado se o agente não começar no ouro
    for step in range(20):
        if state not in q_table:
            q_table[state] = {a: 0 for a in acoes}
        
        # Escolhe a melhor ação aprendida (sem explorar)
        action = max(q_table[state], key=q_table[state].get)

        # Se a ação for atirar
        if 'atirar' in action:
            if not wumpus_morto:
                acertou = atirar(pos, action.split('_')[1])
                if acertou:
                    print("Agente atirou e matou o Wumpus!")
                    wumpus_morto = True
            # O agente não se move ao atirar
            nova_pos = pos
        # Se a ação for mover
        else:
            nova_pos = mover(pos, action)
        
        path.append(nova_pos)
        pos = nova_pos
        state = estado_para_str(pos)

        evento = get_evento(ambiente[pos[0]][pos[1]])
        if (evento == "wumpus" and not wumpus_morto) or evento == "poco" or evento == "ouro":
            break
# --- FIM DA CORREÇÃO ---

print("Caminho seguido pelo agente:", path)

Execução do agente treinado:
Caminho seguido pelo agente: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3)]


No algoritmo atual, estabelecemos as recompensas e o melhor camminho treinando.
Caso seja utilizado o action = get_action(state), teremos resultados mais variados e os diferentes outputs de caminhos seguidos pelo agente após o treinamento acontecem porque o algoritmo Q-learning, por padrão, envolve exploração estocástica (aleatória). Isso significa que mesmo após o treinamento, ainda há uma probabilidade do agente explorar — ou seja, tomar ações não ótimas de propósito.

## Fontes:
- continuam sendo as mesmas fontes do codigo antigo
- https://youtu.be/t_bTU7OqmnA?si=mMCHEi3ArZRsTZSp
- https://github.com/nrupeshsurya/Wumpus-world/blob/master/2017B2A70767G_NRUPESH.py
- chatGPT
- Gemini
- slides da aula